<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Task:-Shopee-binary-sentiment-classification" data-toc-modified-id="Task:-Shopee-binary-sentiment-classification-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Task: Shopee binary sentiment classification</a></span></li><li><span><a href="#Load-data" data-toc-modified-id="Load-data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Load data</a></span><ul class="toc-item"><li><span><a href="#Convert-crash-files-(do-once)" data-toc-modified-id="Convert-crash-files-(do-once)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Convert crash files (do once)</a></span></li><li><span><a href="#Load-csv" data-toc-modified-id="Load-csv-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Load csv</a></span></li></ul></li><li><span><a href="#Vietnamese-tokenization" data-toc-modified-id="Vietnamese-tokenization-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Vietnamese tokenization</a></span><ul class="toc-item"><li><span><a href="#Step-1:-Pretokenization-(Vietnamese-word-tokenization)" data-toc-modified-id="Step-1:-Pretokenization-(Vietnamese-word-tokenization)-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Step 1: Pretokenization (Vietnamese word tokenization)</a></span></li><li><span><a href="#Step-2:-Model's-tokenization" data-toc-modified-id="Step-2:-Model's-tokenization-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Step 2: Model's tokenization</a></span></li></ul></li><li><span><a href="#Use-HuggingFace-Dataset-to-store-and-tokenize-corpus" data-toc-modified-id="Use-HuggingFace-Dataset-to-store-and-tokenize-corpus-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Use HuggingFace Dataset to store and tokenize corpus</a></span></li><li><span><a href="#Load-pretrained-model" data-toc-modified-id="Load-pretrained-model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Load pretrained model</a></span></li></ul></div>

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
from tqdm import tqdm
import re
import os 
from pathlib import Path

In [3]:
# !pip install underthesea

In [4]:
DATA_PATH = Path('data_crash')
RAW_PATH = Path('raw_crash')

# Task: Shopee binary sentiment classification

# Load data

## Convert crash files (do once)

In [5]:


if not os.path.exists(DATA_PATH):
    os.mkdir(DATA_PATH)

### Cleaning training file

train = open(RAW_PATH/"train.crash").readlines()
id_locations = []
label_locations = []
for idx, line in tqdm(enumerate(train)):
    line = line.strip()
    if line.startswith("train_"):
        id_locations.append(idx)
    elif line == "0" or line == "1":
        label_locations.append(idx)
data = []

for id_loc, l_loc in tqdm(zip(id_locations, label_locations)):
    line_id = train[id_loc].strip()
    label = train[l_loc].strip()
    text = re.sub('\s+', ' ', ' '.join(train[id_loc + 1: l_loc])).strip()[1:-1].strip()
    data.append(f"{line_id}\t{text}\t{label}")

with open(DATA_PATH/"train.csv", "wt") as f:
    f.write("id\ttext\tlabel\n")
    f.write("\n".join(data))

### Cleaning test file

test = open(RAW_PATH/"test.crash").readlines()
id_locations = []
for idx, line in tqdm(enumerate(test)):
    line = line.strip()
    if line.startswith("test_"):
        id_locations.append(idx)
data = []

for i, id_loc in tqdm(enumerate(id_locations)):
    if i >= len(id_locations) - 1:
        end = len(test)
    else:
        end = id_locations[i + 1]
    line_id = test[id_loc].strip()
    text = re.sub('\s+', ' ', ' '.join(test[id_loc + 1:end])).strip()[1:-1].strip()
    data.append(f"{line_id}\t{text}")

with open(DATA_PATH/"test.csv", "wt") as f:
    f.write("id\ttext\n")
    f.write("\n".join(data))

66966it [00:00, 1915214.36it/s]
16087it [00:00, 134272.15it/s]
34750it [00:00, 2274852.34it/s]
10981it [00:00, 132888.77it/s]


## Load csv

In [61]:
!ls {str(DATA_PATH)}

test.csv  train.csv


In [6]:
train_df = pd.read_csv(DATA_PATH/'train.csv',sep='\t').fillna("###")
test_df = pd.read_csv(DATA_PATH/'test.csv',sep='\t').fillna("###")

In [7]:
train_df[['text','label']].sample(20)

,text,label
13322,Mẫu mã thì ok nhưng chất lượng ko ổn lắm chỉ g...,1
9448,Giao hàng rất lâu không giống trog ảnh lắm Thờ...,1
9685,Giấy bên trong có kích thước không vừa với kíc...,1
6521,Chất lượng sản phẩm tuyệt vời. Áo mặc rất mát ...,0
4303,Mua của shop lần 2 rồi và lần nào cũng rất hài...,0
7700,Hàng bị bẩn đen làm ăn thế mất uy tín,1
14492,Chất lượng sản phẩm rất kém. Rất không đáng ti...,1
5702,Chất lượng tốt đóng gói đẹp chắc chắn tuy nhiê...,0
14939,Giày bên phải dây dính màu còn giày trái góc t...,1
778,. cũnq thơm mùi hơi nồnq...so vs giá thì oke,0


In [63]:
print(train_df[['text','label']].sample(5).values)

[['Chất lượng sản phẩm tuyệt vời và đang mình đặt 1 chiếc nữa' 0]
 ['Cảm thấy bị lừa đảo' 1]
 ['Sach rat dep' 0]
 ['Chất lượng sản phẩm tuyệt vời. Rất đáng tiền' 0]
 ['Rất không đáng tiền. Đặt số 42 hết số kiếm đâu đôi 40 dán đại cai số 42 rồi giao cho khách hàng. K biết làm an kiểu j. Hàng thì k cho xem k cho kiểm tra. Bắt phải nhận mới đc mở. Ép người ta lấy hàng bán k đc. Đủ trò.'
  1]]


In [9]:
print(test_df[['text']].sample(5).values)

[['Chất lượng sản phẩm tuyệt vời. Dán cho con s8+ xong nhìn sướng cả mắt. Trc dùng dán dẻo toàn bị mất 1 tý rìa màn hình']
 ['Chất lượng sản phẩm tuyệt vời.Tuyet vời']
 ['Đóng gói sản phẩm rất rất chắc chắn. Giao hàng nhanh. Đúng màu. Còn có cả phiếu bảo hành ^^ Tuy chưa có điều kiện dùng thử nhưng thấy ấn tượng đầu rất tốt. Chúc shop bán đắt nha. 10 điểm cho chất lượng và tác phong bán hàng nek. Yêu shop <3 Rất đáng tiền']
 ['Đẹp lung linh']
 ['4sim 2 thẻ nhớ 8 sóng 12 mạng :):) đã test thử ukie màu kute phô mai qe']]


# Vietnamese tokenization

![](https://learning.oreilly.com/api/v2/epubs/urn:orm:book:9781098136789/files/assets/nlpt_0401.png)

Jack Sparrow loves New York!


1. Normalization
    - set of operations you apply to a raw string to **make it “cleaner”**, e.g. stripping whitespace, rm accented chars, lowercasing, Unicode normalization (unify various ways to write the same character)

=> jack sparrow loves new york!

2. Pretokenization
    - splits a text into smaller objects (can be words) that **give an upper bound to what your tokens will be at the end of training; your final tokens will be parts of these smaller objects**
    - Sometimes splitting into 'words' is not always trivial (Chinese, Japanese, Korean). In this case, it might be best to not pretokenize the text and instead use a language-specific library for pretokenization.

=> ["jack", "sparrow", "loves", "new", "york", "!"]

3. Tokenizer model
    - tokenizer applies a **subword splitting model** on the words. This is the part of the pipeline that **needs to be trained on your corpus (or that has been trained if you are using a pretrained tokenizer)**
    -  to split the words into subwords to reduce the size of the vocabulary and try to reduce the number of out-of-vocabulary tokens
    - Several subword tokenization algorithms exist, including BPE, Unigram, and WordPiece
    
=> [jack, spa, rrow, loves, new, york, !]

NOTE: at this point we no longer have a list of strings but a list of integers (input IDs)

4. Postprocessing
    - some additional transformations can be applied on the list of tokens
    - e.g. adding special tokens at the beginning or the end
    - This is the last step, and the sequence of integers can be fed to the model
=> a BERT-style tokenizer would add classifications and separator tokens: [CLS, jack, spa, rrow, loves, new, york, !, SEP]
    

## Step 1: Pretokenization (Vietnamese word tokenization)

https://github.com/undertheseanlp/underthesea

```python
>>> from underthesea import word_tokenize
>>> sentence = 'Chàng trai 9X Quảng Trị khởi nghiệp từ nấm sò'

>>> word_tokenize(sentence)
['Chàng trai', '9X', 'Quảng Trị', 'khởi nghiệp', 'từ', 'nấm', 'sò']

>>> word_tokenize(sentence, format="text")
'Chàng_trai 9X Quảng_Trị khởi_nghiệp từ nấm sò'
```

In [10]:
from underthesea import word_tokenize, sent_tokenize

In [11]:
def apply_word_tokenize(sen,split_word=False):
    # optional step: fix the whitespace between words
    sen = " ".join(sen.split())
    sens = sent_tokenize(sen)
    
    # word tokenize
    tokenized_sen = []
    for sen in sens:
        tokenized_sen+=word_tokenize(sen,format='text' if not split_word else None)
    
    if not split_word:
        return ''.join(tokenized_sen)
    return ['_'.join(words.split(' ')) for words in tokenized_sen]
        

In [64]:
_tmp = 'Chất lượng sản phẩm tuyệt vời, phấn mịn, đóng gói sản phẩm rất đẹp và chắc chắn'
# print(apply_word_tokenize(_tmp,False))
print(apply_word_tokenize(_tmp,True))

['Chất_lượng', 'sản_phẩm', 'tuyệt_vời', ',', 'phấn', 'mịn', ',', 'đóng_gói', 'sản_phẩm', 'rất', 'đẹp', 'và', 'chắc_chắn']


In [66]:
_tmp = 'Chất lượng sản phẩm tuyệt vời? Phấn mịn, đóng gói sản phẩm rất đẹp và chắc chắn'
# print(apply_word_tokenize(_tmp,False))
print(apply_word_tokenize(_tmp,True))

['Chất_lượng', 'sản_phẩm', 'tuyệt_vời', '?', 'Phấn', 'mịn', ',', 'đóng_gói', 'sản_phẩm', 'rất', 'đẹp', 'và', 'chắc_chắn']


In [69]:
_tmp = 'Chất lượng sản phẩm tuyệt vời 😌😌. Phấn mịn, đóng gói sản phẩm rất đẹp và chắc chắn'
# print(apply_word_tokenize(_tmp,False))
print(apply_word_tokenize(_tmp,True))

['Chất_lượng', 'sản_phẩm', 'tuyệt_vời', '😌_😌', '.', 'Phấn', 'mịn', ',', 'đóng_gói', 'sản_phẩm', 'rất', 'đẹp', 'và', 'chắc_chắn']


Apply on the whole dataset

In [16]:
train_text = [apply_word_tokenize(s[0],True) for s in train_df[['text']].values]
train_label = train_df.label.values

In [17]:
test_text = [apply_word_tokenize(s[0],True) for s in test_df[['text']].values]

In [18]:
train_text[10:15]

[['Chất_lượng',
  'sản_phẩm',
  'tuyệt_vời',
  '.',
  'y',
  'hình',
  'chụp',
  '.',
  'đáng',
  'tiền'],
 ['Hjhj_shop',
  'giao',
  'hàng',
  'nhanh',
  'quá',
  '.',
  'Đẹp',
  'lắm',
  'ạ',
  'bé',
  'nhà',
  'm',
  'rất',
  'thích'],
 ['nhìn', 'đẹp', 'phết', 'nhỉ', '..'],
 ['Đóng_gói',
  'rất',
  'đẹp',
  '.',
  'Chất_lượng',
  'sản_phẩm',
  'rất',
  'tốt',
  'Chất_lượng',
  'sản_phẩm',
  'tuyệt_vời'],
 ['Săn', 'đc', 'với', 'giá', '11', 'k', '.', 'Toẹt', 'vời']]

## Step 2: Model's tokenization

To use pretrained language model such as BERT, GPT, Roberta... We need to tokenize words using the strategy in these models (BPE, wordpiece, ...)

Huggingface allows us to get the tokenizer corresponding to the model by the model name on their hub. In this notebook, we use PhoBERT-base (vinai/phobert-base).

In [22]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")# model name in huggingface's hub


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [24]:
def tokenizer_explain(inp,split_word):
    print('--- Tokenized results --- ')
    print(tokenizer(inp,is_split_into_words=split_word))
    print()
    tok = tokenizer.encode(inp,is_split_into_words=split_word)
    print('--- Results from tokenizer.convert_ids_to_tokens')
    print(tokenizer.convert_ids_to_tokens(tok))
    print()
    print('--- Results from tokenizer.decode --- ')
    print(tokenizer.decode(tok))
    print()


def two_step_tokenization_explain(inp,split_word=False):
    print('--- Raw sentence ---')
    print(inp)
    print()
    print('--- Pretokenization ---')
    tok = apply_word_tokenize(inp,split_word)
    print(tok)
    print()
    tokenizer_explain(tok,split_word)

In [26]:
_tmp = 'Chất lượng sản phẩm tuyệt vời, phấn mịn, đóng gói sản phẩm rất đẹp và chắc chắn'
two_step_tokenization_explain(_tmp,True)

--- Raw sentence ---
Chất lượng sản phẩm tuyệt vời, phấn mịn, đóng gói sản phẩm rất đẹp và chắc chắn

--- Pretokenization ---
['Chất_lượng', 'sản_phẩm', 'tuyệt_vời', ',', 'phấn', 'mịn', ',', 'đóng_gói', 'sản_phẩm', 'rất', 'đẹp', 'và', 'chắc_chắn']

--- Tokenized results --- 
{'input_ids': [0, 6869, 265, 1819, 4, 7079, 5451, 4, 8179, 265, 59, 258, 6, 994, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

--- Results from tokenizer.convert_ids_to_tokens
['<s>', 'Chất_lượng', 'sản_phẩm', 'tuyệt_vời', ',', 'phấn', 'mịn', ',', 'đóng_gói', 'sản_phẩm', 'rất', 'đẹp', 'và', 'chắc_chắn', '</s>']

--- Results from tokenizer.decode --- 
<s> Chất_lượng sản_phẩm tuyệt_vời, phấn mịn, đóng_gói sản_phẩm rất đẹp và chắc_chắn </s>



In [71]:
_tmp = 'Chất lượng sản phẩm tuyệt vời. Phấn mịn, đóng gói sản phẩm rất đẹp và chắc chắn'
two_step_tokenization_explain(_tmp,split_word=False)

--- Raw sentence ---
Chất lượng sản phẩm tuyệt vời. Phấn mịn, đóng gói sản phẩm rất đẹp và chắc chắn

--- Pretokenization ---
Chất_lượng sản_phẩm tuyệt_vời .Phấn mịn , đóng_gói sản_phẩm rất đẹp và chắc_chắn

--- Tokenized results --- 
{'input_ids': [0, 6869, 265, 1819, 2586, 11459, 5451, 4, 8179, 265, 59, 258, 6, 994, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

--- Results from tokenizer.convert_ids_to_tokens
['<s>', 'Chất_lượng', 'sản_phẩm', 'tuyệt_vời', '.@@', 'Phấn', 'mịn', ',', 'đóng_gói', 'sản_phẩm', 'rất', 'đẹp', 'và', 'chắc_chắn', '</s>']

--- Results from tokenizer.decode --- 
<s> Chất_lượng sản_phẩm tuyệt_vời.Phấn mịn, đóng_gói sản_phẩm rất đẹp và chắc_chắn </s>



In [28]:
_tmp = 'Chất lượng sản phẩm tuyệt vời. Phấn mịn, đóng gói sản phẩm rất đẹp và chắc chắn'
two_step_tokenization_explain(_tmp,True)

--- Raw sentence ---
Chất lượng sản phẩm tuyệt vời. Phấn mịn, đóng gói sản phẩm rất đẹp và chắc chắn

--- Pretokenization ---
['Chất_lượng', 'sản_phẩm', 'tuyệt_vời', '.', 'Phấn', 'mịn', ',', 'đóng_gói', 'sản_phẩm', 'rất', 'đẹp', 'và', 'chắc_chắn']

--- Tokenized results --- 
{'input_ids': [0, 6869, 265, 1819, 5, 11459, 5451, 4, 8179, 265, 59, 258, 6, 994, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

--- Results from tokenizer.convert_ids_to_tokens
['<s>', 'Chất_lượng', 'sản_phẩm', 'tuyệt_vời', '.', 'Phấn', 'mịn', ',', 'đóng_gói', 'sản_phẩm', 'rất', 'đẹp', 'và', 'chắc_chắn', '</s>']

--- Results from tokenizer.decode --- 
<s> Chất_lượng sản_phẩm tuyệt_vời. Phấn mịn, đóng_gói sản_phẩm rất đẹp và chắc_chắn </s>



In [30]:
_tmp = 'Chất lượng sản phẩm tuyệt vời 😌😌. Phấn mịn, đóng gói sản phẩm rất đẹp và chắc chắn'
two_step_tokenization_explain(_tmp,split_word=True)

--- Raw sentence ---
Chất lượng sản phẩm tuyệt vời 😌😌. Phấn mịn, đóng gói sản phẩm rất đẹp và chắc chắn

--- Pretokenization ---
['Chất_lượng', 'sản_phẩm', 'tuyệt_vời', '😌_😌', '.', 'Phấn', 'mịn', ',', 'đóng_gói', 'sản_phẩm', 'rất', 'đẹp', 'và', 'chắc_chắn']

--- Tokenized results --- 
{'input_ids': [0, 6869, 265, 1819, 3, 1751, 3, 5, 11459, 5451, 4, 8179, 265, 59, 258, 6, 994, 2], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

--- Results from tokenizer.convert_ids_to_tokens
['<s>', 'Chất_lượng', 'sản_phẩm', 'tuyệt_vời', '<unk>', '_@@', '<unk>', '.', 'Phấn', 'mịn', ',', 'đóng_gói', 'sản_phẩm', 'rất', 'đẹp', 'và', 'chắc_chắn', '</s>']

--- Results from tokenizer.decode --- 
<s> Chất_lượng sản_phẩm tuyệt_vời <unk> _<unk>. Phấn mịn, đóng_gói sản_phẩm rất đẹp và chắc_chắn </s>



# Use HuggingFace Dataset to store and tokenize corpus

In [52]:
from datasets import DatasetDict,Dataset

In [76]:
train_dataset = Dataset.from_dict(
                        {'text': train_text[:4],
                        'label':train_label[:4],
                        }
                    )
# test_dataset = Dataset.from_dict(
#                         {'text': test_text,
#                         }
#                     )

In [77]:
def tokenize_function(examples):
    # pad to model's allowed max length, which is max_sequence_length
    return tokenizer(examples["text"], padding="max_length", truncation=True,is_split_into_words=True)


In [78]:
train_dataset_tokenized = train_dataset.map(tokenize_function,batched=True,batch_size=2)

  0%|          | 0/2 [00:00<?, ?ba/s]

In [79]:
train_dataset_tokenized

Dataset({
    features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 4
})

In [80]:
print(train_dataset_tokenized['text'])

[['Dung', 'dc', 'sp', 'tot', 'cam', 'on_shop', 'Đóng_gói', 'sản_phẩm', 'rất', 'đẹp', 'và', 'chắc_chắn', 'Chất_lượng', 'sản_phẩm', 'tuyệt_vời'], ['Chất_lượng', 'sản_phẩm', 'tuyệt_vời', '.', 'Son', 'mịn', 'nhưng', 'khi', 'đánh', 'lên', 'không', 'như', 'màu', 'trên', 'ảnh'], ['Chất_lượng', 'sản_phẩm', 'tuyệt_vời', 'nhưng', 'k', 'có', 'hộp', 'k', 'có', 'dây', 'giày', 'đen', 'k', 'có', 'tất'], [':', '(', '(', 'Mình', 'hơi', 'thất_vọng', '1', 'chút', 'vì', 'mình', 'đã', 'kỳ_vọng', 'cuốn', 'sách', 'khá', 'nhiều', 'hi_vọng', 'nó', 'sẽ', 'nói', 'về', 'việc', 'học_tập', 'của', 'cách', 'sinh_viên', 'trường', 'Harvard', 'ra_sao', 'những', 'nỗ_lực', 'của', 'họ', 'như', 'thế_nào', '4', 'h', 'sáng', '?', 'tại_sao', 'họ', 'lại', 'phải', 'thức', 'dậy', 'vào', 'thời_khắc', 'đấy', '?', 'sau', 'đó', 'là', 'cả', 'một', 'câu_chuyện', 'ra_sao', '.', 'Cái', 'mình', 'thực_sự', 'cần', 'ở', 'đây', 'là', 'câu_chuyện', 'ẩn', 'dấu', 'trong', 'đó', 'để', 'tự', 'bản_thân', 'mỗi', 'người', 'cảm_nhận', 'và', 'đi', 'sâu

In [83]:
print(train_dataset_tokenized['input_ids'])

[[0, 3556, 1236, 1894, 36150, 2225, 1204, 2947, 1672, 20811, 54922, 55662, 1685, 265, 59, 258, 6, 994, 6869, 265, 1819, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 6869, 265, 1819, 5, 16332, 5451, 51, 26, 480, 72, 17, 42, 412, 34, 284, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

# Load pretrained model

In [33]:
from transformers import AutoModelForSequenceClassification

In [34]:
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=2)

Downloading:   0%|          | 0.00/518M [00:00<?, ?B/s]

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['